<a href="https://colab.research.google.com/github/MHHamdan/LLM_Reasoning/blob/main/Search_reranking_using_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [ ]:
!pip install -q -U "google-generativeai>=0.7.2"

In [ ]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [ ]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-1.5-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [ ]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-1.5-flash')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-1.5-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [ ]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Stochastic parrot', 'Artificial intelligence content detection']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Stochastic parrot"
Information Source: https://en.wikipedia.org/wiki/Stochastic_parrot


Stochastic parrot did not match with any page id, hence skipping.
Fetching page: "Artificial intelligence content detection"
Information Source: https://en.wikipedia.org/wiki/Artificial_intelligence_content_detection
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Stochastic_parrot
     https://en.wikipedia.org/wiki/Artificial_intelligence_content_detection


Here is what the search results look like:

In [ ]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> LLMs, or large language models, are machine learning models designed for natural language processing tasks like language generation.  They have many parameters and are trained with self-supervised learning on vast amounts of text.  The most capable LLMs are generative pretrained transformers (GPTs).  Modern models can be fine-tuned for specific tasks or guided by prompt engineering.  They acquire predictive power regarding syntax, semantics, and ontologies in human language but also inherit inaccuracies and biases from their training data.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> Large language models (LLMs) are mentioned in relation to watermarking techniques.  A 2023 paper, "A Watermark for Large Language Models," describes a method to embed imperceptible watermarks into text generated by LLMs.  LLMs like ChatGPT, Claude, and Gemini are also noted for their ability to write in different languages, highlighting a shortcoming of AI detection software which is primarily trained in English and a few other languages.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Artificial_intelligence_content_detection

### Pass the tools to the model

If you pass a list of functions to the `GenerativeModel`'s `tools` argument,
it will extract a schema from the function's signature and type hints, and then pass schema along to the API calls. In response the model may return a `FunctionCall` object asking to call the function.

Note: This approach only handles annotations of `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

The `GenerativeModel` will keep a reference to the function inself, so that it _can_ execute the function locally later.

In [ ]:
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [ ]:
model = genai.GenerativeModel(
    'gemini-1.5-flash', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How do deep sea creatures survive?"
Related search terms: ['Deep-sea community', 'Deep sea', 'Still Wakes the Deep']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Still Wakes the Deep"
Information Source: https://en.wikipedia.org/wiki/Still_Wakes_the_Deep
Searching for "What adaptations do deep sea animals have?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Cyclothone']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Cyclothone"
Information Source: https://en.wikipedia.org/wiki/Cyclothone
Searching for "How do deep sea animals cope with pressure?"
Related search terms: ['Deep sea', 'Deep-sea community', 'Deep-sea fish']
Searching for "How do deep sea animals find food?"
Related search ter

Deepsea Challenger did not match with any page id, hence skipping.
Fetching page: "Deep-sea exploration"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_exploration
Searching for "Deep sea hydrothermal vents"
Related search terms: ['Hydrothermal vent', 'Hydrothermal vent microbial communities', 'Scaly-foot gastropod']
Fetching page: "Hydrothermal vent microbial communities"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent_microbial_communities
Fetching page: "Scaly-foot gastropod"
Information Source: https://en.wikipedia.org/wiki/Scaly-foot_gastropod
Scaly-foot gastropod did not match with any page id, hence skipping.
Searching for "Deep sea extremophiles"
Related search terms: ['Hydrothermal vent', 'Deep-sea community', 'Extremophile']
Fetching page: "Extremophile"
Information Source: https://en.wikipedia.org/wiki/Extremophile
Extremophile did not match with any page id, hence skipping.
Information Sources:
     https://en.wikipedia.org/wiki/Deep-sea_comm

In [ ]:
to_markdown(res.text)

> Deep-sea life faces extreme conditions: crushing pressure, freezing temperatures, and total darkness.  Organisms have adapted in remarkable ways to survive.  Many are small and gelatinous to withstand pressure, relying on marine snow (sinking organic matter) or chemosynthesis (using chemicals for energy) near hydrothermal vents for food.  Bioluminescence (creating light) is common for attracting prey or mates.  Some animals have symbiotic relationships with bacteria, providing energy.  In short, deep-sea survival hinges on adaptations to pressure, food scarcity, darkness, and extreme temperatures, often involving unique biological strategies and partnerships.


Check for additional citations:

In [ ]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [ ]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

In the chat history you can see all 4 steps:

1. The user sent the query.
2. The model replied with a `genai.protos.FunctionCall` calling the `wikipedia_search` with a number of relevant searches.
3. Because you set `enable_automatic_function_calling=True` when creating the `genai.ChatSession`, it  executed the search function and returned the list of article summaries to the model.
4. Folliwing the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [ ]:
chat = model.start_chat()

In [ ]:
result = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [ ]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep sea creatures survive?",
      "What are the adaptations of deep-sea life?",
      "How do deep-sea organisms cope with pressure?",
      "How do deep-sea organisms obtain food?",
      "Deep sea life survival strategies",
      "Deep sea ecosystem",
      "Adaptations of deep sea animals",
      "Challenges of deep sea survival"
    ]
  }
}


In [ ]:
fc['name']

'wikipedia_search'

Call the function with generated arguments to get the results.

In [ ]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep sea creatures survive?"
Related search terms: ['Deep-sea community', 'Deep sea', 'Still Wakes the Deep']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Still Wakes the Deep"
Information Source: https://en.wikipedia.org/wiki/Still_Wakes_the_Deep
Searching for "What are the adaptations of deep-sea life?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Cyclothone']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Cyclothone"
Information Source: https://en.wikipedia.org/wiki/Cyclothone
Searching for "How do deep-sea organisms cope with pressure?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Searching for "How do deep-sea organisms obtain food?"
Related search terms: ['Hydrothermal vent', 'Deep-sea community', 'Fo

Now send the `FunctionResult` to the model.

In [ ]:
response = chat.send_message(
    genai.protos.Content(
      parts=[genai.protos.Part(
          function_response = genai.protos.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea life faces extreme pressure, cold temperatures, and darkness.  Organisms have adapted in various ways to survive.  Many are small, with gelatinous bodies to withstand pressure.  They often have slow metabolisms and feed on whatever limited resources are available, such as marine snow (organic matter falling from above), or chemosynthesis around hydrothermal vents.  Some have bioluminescence for attracting prey or mates.  Many have adaptations for pressure tolerance, including specialized proteins and cell membranes.  The specifics vary greatly depending on the species and depth.


## Re-ranking the search results

Helper function to embed the content:

In [ ]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [ ]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [ ]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
print(summaries[np.argmax(sim_value)])

This document is about a video game and does not contain information about how deep sea creatures survive.


Based on:
  https://en.wikipedia.org/wiki/Still_Wakes_the_Deep


### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-1.5-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [ ]:
hypothetical_ans_model = genai.GenerativeModel('gemini-1.5-flash')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> Oh wow, deep-sea life!  It's amazing how they manage it, isn't it?  It's a completely different world down there.  Essentially, they've adapted to overcome three major challenges:  the crushing pressure, the absolute lack of sunlight, and the incredibly scarce food supply.
> 
> The pressure thing is solved with some seriously clever biochemistry.  Their bodies are essentially built with [Specific biochemical adaptation 1] and [Specific biochemical adaptation 2] that allow their cells and tissues to withstand the immense pressure without imploding.  Think of it like having a super-strong, flexible suit of armor built right into your cells.
> 
> The lack of sunlight?  That's where things get *really* interesting. Forget photosynthesis –  most deep-sea creatures rely on [Alternative energy source 1] like chemosynthesis in hydrothermal vents, or they've developed incredible sensory systems to detect even the faintest [Specific sensory stimuli] to find food. Some species even engage in [Unique feeding strategy] to maximize their meager resources.
> 
> And the food?  Let's just say it's a "feast or famine" existence.  They have incredibly efficient metabolisms, often slowing down their [Metabolic function] to conserve energy between meals, which might be days, weeks, or even months apart.  They've also evolved [Adaptation for scavenging/predation] to make the most of whatever limited food does drift down from the surface – even if it's just a tiny bit of [Type of organic matter].
> 
> So, in short, deep-sea life isn't just surviving, it's thriving... in its own incredibly unique and adapted way. They've essentially become masters of efficiency, utilizing every available resource and adapting their very biology to overcome the extreme challenges of their environment. It’s truly a testament to the incredible resilience of life.


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [ ]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

array([[0.85123131],
       [0.84652198],
       [0.72562176],
       [0.7969651 ],
       [0.81950443],
       [0.8053952 ],
       [0.7567845 ],
       [0.71918985],
       [0.75196504],
       [0.8035631 ],
       [0.76306842],
       [0.77025518],
       [0.76567088]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
to_markdown(summaries[np.argmax(sim_value)])

> Deep-sea creatures survive by adapting to the extreme pressure, temperature, and darkness of their environment.  Adaptations include: small size (generally not exceeding 25 cm), gelatinous flesh, minimal skeletal structure, elimination of cavities (like swim bladders) to withstand pressure; and tolerance for cold, constant temperatures (below the thermocline)  or extreme temperature fluctuations near hydrothermal vents.  Food sources include marine snow, occasional surface blooms of plankton, whale falls, and chemosynthesis at hydrothermal vents and cold seeps.  Chemosynthetic bacteria provide energy at vents and seeps, forming the base of the food web.  Many organisms in the mesopelagic and bathypelagic zones make vertical migrations to feed, utilizing swim bladders (though some have lost them).  In the aphotic zone, organisms rely on material sinking from above or chemosynthesis.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_community

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.